<img src="https://www.cecodes.org.co/site/wp-content/uploads/2016/01/WP_asociado_COLSUBSIDIO-832x350.jpg" alt="Logo Colsubsidio" width=200>

In [1]:
# conda install -c r rpy2
%load_ext rpy2.ipython 
import os
import numpy as np
import pandas as pd
import re
from datetime import datetime

In [2]:
import unidecode #conda install -c anaconda unidecode
def Limpiar_Nombres(cadena):
    s=re.sub(' ','',re.sub(r'(\W)\1+', r'\1',cadena.lower()))
    s=unidecode.unidecode(s)
    return s

# Calificación Modelo de Retención

Para calificar la base se necesita la base de datos de cobranzas el primer dia hábil del mes en formato **csv**

## Importación

Para la calificación del modelo se requieren 3 bases de datos:

1. Base de datos de cobranza a cierre del mes de análisis: \\BOGA04beimrodc\FILIALES\Credito\BD Cobranzas
2. Listados de identificaciones con el fin de homologar con el formato de identificación de Inteligencia de Negocios.
3. Listados de estados de tarjetas.

### Base de datos de cobranzas

In [3]:
calif = pd.read_csv("../BD/Calificar/Base_Calificar.csv", encoding='latin1', index_col=False, low_memory=False, thousands=',')
calif = calif.rename(columns=lambda x: Limpiar_Nombres(x))
calif=calif[["tipoidentificacion","nroidentificacion","tarjeta","fechaexpedicion","fechaproceso","disponibleavances",
             "limiteavances","totalintereses","saldosmesant","pagosmesant","vtasmesant","edadmora","limitecupo",
             "pagodelmes","pagominimo","vrmora","vrcuotamanejo","saldo","estadotarjeta"]]

Antes de continuar es importante verificar que la fecha de proceso de la base cargada corresponda a la que se requiere

In [4]:
print("Base de datos con fecha: ",list(set(calif['fechaproceso'])))

Base de datos con fecha:  ['01/08/2018']


### Base de datos de identificaciones

In [5]:
id_ = pd.DataFrame(data={'tipoidentificacion': [1,2,3,4], 't_id': ["","CC","CE","TI"]})

### Base de Datos de Estados de tarjeta

In [6]:
estados = pd.read_csv("../BD/Auxiliar/EstadosTarjeta.csv", encoding='latin1', index_col=False, sep=";")

## Preparación de los datos

Inicialmente se construye una base de datos que contiene las caracteristicas para las tarjetas activas a la fecha de corte

In [7]:
# Unión de las bases de datos.
DB=calif.merge(id_,on='tipoidentificacion', how='left').merge(estados,on='estadotarjeta', how='left')

# Filtro a cuentas activas
DB=DB[DB['ESTADOCUPO']=='ACTIVO']

# Construcción ID formato inteligencia de negocios.
DB['IdPersona']= DB["t_id"].map(str) + DB["nroidentificacion"].map(str)

# Selección de Columnas
DB=DB[['IdPersona', "tarjeta", "fechaexpedicion","fechaproceso","disponibleavances","limiteavances", "totalintereses",
       "saldosmesant","pagosmesant","vtasmesant","edadmora","limitecupo","pagodelmes","pagominimo","vrmora","vrcuotamanejo",
       "saldo"]]

# Renombramiento de columnas para que coincidan con el modelo
DB.columns=["identificacion","tarjeta","fecha_expedicion","fecha_proceso","disponible_avances",
                "limite_avances","total_intereses","saldos_mes_ant","pagos_mes_ant","vtas_mes_ant","edad_mora",
                "limite_cupo","pago_del_mes","pago_minimo","vr_mora","vr_cuota_manejo","saldo"]

Adicional a la unión de los conjuntos de datos es necesario construir ciertas variables explicativas. Además s modificarán los formatos de ciertas variables

In [8]:
# Modificación de tipos de variable 
cate=["identificacion","tarjeta"]
nume=["edad_mora"]
fech=["fecha_expedicion", "fecha_proceso"]

for var in cate:
    DB[var] = DB[var].astype("category")
    
for var in nume:
    DB[var] = DB[var].astype("float64")
    
for var in fech:
    DB[var]=pd.to_datetime(DB[var], format="%d/%m/%Y")

# Construcción de nuevas variables 
DB['dias_intencion']=(DB['fecha_proceso']-DB['fecha_expedicion'])/np.timedelta64(1, 'D')

# Verificacion
DB.head(3)

,identificacion,tarjeta,fecha_expedicion,fecha_proceso,disponible_avances,limite_avances,total_intereses,saldos_mes_ant,pagos_mes_ant,vtas_mes_ant,edad_mora,limite_cupo,pago_del_mes,pago_minimo,vr_mora,vr_cuota_manejo,saldo,dias_intencion
326,CC41668353,52898,2012-08-16,2018-08-01,3970000.0,3970000.0,0.0,25161.65,25325.0,18117.0,0.0,7940000.0,25162.0,0.0,0.0,0.0,18116.65,2176.0
428,CC79743393,69421,2006-11-03,2018-08-01,150000.0,150000.0,0.0,0.00,0.0,0.0,0.0,500000.0,0.0,0.0,0.0,0.0,0.00,4289.0
528,CC79415249,91862,2007-06-27,2018-08-01,1200000.0,1200000.0,0.0,0.00,0.0,0.0,0.0,3000000.0,0.0,0.0,0.0,0.0,0.00,4053.0


#### Selección de Variables Predictoras

In [9]:
X = DB[['disponible_avances', 'limite_avances','total_intereses', 'saldos_mes_ant', 'pagos_mes_ant', 'vtas_mes_ant',
          'edad_mora', 'limite_cupo', 'pago_del_mes', 'pago_minimo', 'vr_mora','vr_cuota_manejo', 'saldo','dias_intencion']]

## Carga del Modelo

In [10]:
from sklearn.externals import joblib
model = joblib.load('../Resultados/ModeloRetencion.pkl') 

## Calificación de los datos

In [17]:
Prob = model.predict_proba(X)
DB['Prob1']=Prob[0:,1]*1000

## Preparación entrega de Base

A partir de las calificaciones se construye la categorización del score de fuga con los siguientes parámetros

|   Rango  	| Mínimo 	| Máximo 	|
|:--------:	|:------:	|:------:	|
| Muy Bajo 	|    0   	|   299  	|
| Bajo     	|   300  	|   599  	|
| Medio    	|   600  	|   899  	|
| Alto     	|   900  	|  1000  	|


In [18]:
DB['Score Agrupado'] = np.where(DB.Prob1<=229, 'Muy Bajo',
                                np.where(DB.Prob1<=599, 'Muy Bajo',
                                         np.where(DB.Prob1<=899, 'Muy Bajo','Alto')))

In [23]:
DB.columns.values

array(['identificacion', 'tarjeta', 'fecha_expedicion', 'fecha_proceso',
       'disponible_avances', 'limite_avances', 'total_intereses',
       'saldos_mes_ant', 'pagos_mes_ant', 'vtas_mes_ant', 'edad_mora',
       'limite_cupo', 'pago_del_mes', 'pago_minimo', 'vr_mora',
       'vr_cuota_manejo', 'saldo', 'dias_intencion', 'Prob1',
       'Score Agrupado'], dtype=object)

In [29]:
BD_Final=DB[['identificacion', 'tarjeta', 'Score Agrupado', 'Prob1']]
BD_Final.columns=['Identificación','Tarjeta','Score Agrupado','Score']
BD_Final.head(3)

,Identificación,Tarjeta,Score Agrupado,Score
326,CC41668353,52898,Muy Bajo,132.0
428,CC79743393,69421,Muy Bajo,0.0
528,CC79415249,91862,Muy Bajo,0.0


## Exportación de la base

In [32]:
writer = pd.ExcelWriter('../BD/Resultados/Calificadas/Jul2018.xlsx')
BD_Final.to_excel(writer,'Sheet1', index=False)
writer.save()